# Face Recognition dengan Akurasi
## Menggunakan FaceNet (VGGFace2) + Augmentation + MTCNN Alignment  
### Dataset dari Roboflow

**Tujuan notebook ini:**
- Download dataset face recognition dari Roboflow (format folder)
- Training FaceNet dengan augmentasi
- Tuning threshold otomatis
- Testing + visualisasi

---

## 1. Install & Import Library

In [1]:
# Install library yang dibutuhkan
!pip install -q roboflow facenet_pytorch opencv-python-headless albumentations insightface mtcnn tqdm

# Import semua library
import os
import torch
import numpy as np
from PIL import Image
import cv2
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

print("Semua library berhasil di-import!")

Semua library berhasil di-import!


## 2. Download Dataset dari Roboflow

In [2]:
!pip install -q roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="QOd5ldAdjiaehHn5m6WC")
project = rf.workspace("dentalogic8").project("face-sz6g1")
version = project.version(1)
# =================================================================

# Download dalam format "folder" (paling cocok untuk face recognition)
dataset = version.download("folder")

# Cek lokasi dataset
dataset_location = dataset.location
print(f"\nDataset berhasil di-download ke:\n {dataset_location}\n")
print("Isi folder train:")
!ls -l "{dataset_location}/train" | head -20

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Face-1 in folder:: 100%|██████████| 824/824 [00:00<00:00, 13177.77it/s]



Dataset berhasil di-download ke:
 /content/Face-1

Isi folder train:
total 220
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Aan Kristian Sitinjak
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Abeloisa Pardosi
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Adriano Lumbantoruan
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Agnes Sidabutar
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Andri Sigiro
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Angelika Simamora
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Anno Siregar
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Antonia Manalu
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Arnold Daniel Manalu
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Ayu Sinaga
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Bowo Manalu
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Chelsea Sianipar
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Chenit Sirait
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Cheryl
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Daniel Ginting
drwxr-xr-x 2 root root 4096 Nov 30 15:01 Diva Marbun
drwxr-xr-x 2 root ro

## 3. Konfigurasi Utama

In [3]:
# ------------------- CONFIG (SESUAIKAN) -------------------
DATASET_ROOT = f"{dataset.location}/train"   # Folder train dari dataset Roboflow
RESULT_FOLDER = "hasil_validasi"
os.makedirs(RESULT_FOLDER, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device yang digunakan: {DEVICE}")

# Augmentasi berat per foto (semakin tinggi = semakin akurat, makin lama juga)
AUG_PER_IMAGE = 50
USE_MTCNN_ALIGN = True    # True = paling akurat (alignment berdasarkan landmark)
USE_HAAR_CROP = True      # Kalau MTCNN gagal

Device yang digunakan: cuda


## 4. Haar Cascade (Fallback Face Detection)

In [4]:
# Load Haar Cascade untuk crop wajah manual (cadangan)
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def crop_face_haar(img_cv2):
    gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100,100))
    if len(faces) > 0:
        x, y, w, h = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)[0]
        margin = int(w * 0.3)
        x = max(x - margin, 0)
        y = max(y - margin, 0)
        w += 2 * margin
        h += 2 * margin
        return img_cv2[y:y+h, x:x+w]
    return None

## 5. MTCNN untuk Face Alignment

In [5]:
# MTCNN untuk alignment wajah (160x160) - sangat meningkatkan akurasi
mtcnn = MTCNN(image_size=160, margin=0, keep_all=False, device=DEVICE) if USE_MTCNN_ALIGN else None

## 6. Augmentation dengan Albumentations

In [6]:
aug_heavy = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=25, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=30, p=0.7),
    A.GaussNoise(var_limit=(10, 50), p=0.4),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
    A.RandomGamma(gamma_limit=(70, 130), p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.4),
    A.GaussianBlur(blur_limit=(3,7), p=0.3),
    A.MotionBlur(blur_limit=7, p=0.2),
    A.RandomShadow(p=0.3),
    A.RandomFog(p=0.2),
    A.RandomRain(p=0.1),
    A.CLAHE(clip_limit=3.0, p=0.5),
    A.Solarize(threshold=128, p=0.2),
    A.Posterize(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, border_mode=0, p=0.8),
], p=1.0)

transform_to_tensor = ToTensorV2()

Argument(s) 'var_limit' are not valid for transform GaussNoise
Argument(s) 'threshold' are not valid for transform Solarize
ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.


## 7. Load Model FaceNet (Pretrained VGGFace2)

In [7]:
print("Loading FaceNet (InceptionResnetV1) pretrained on VGGFace2...")
model = InceptionResnetV1(pretrained='vggface2').eval().to(DEVICE)

Loading FaceNet (InceptionResnetV1) pretrained on VGGFace2...


  0%|          | 0.00/107M [00:00<?, ?B/s]

## 8. Fungsi Ekstraksi Embedding (dengan Fallback)

In [8]:
def get_embedding(img_pil_or_cv2):
    # Konversi ke PIL jika input dari OpenCV
    if isinstance(img_pil_or_cv2, np.ndarray):
        img = Image.fromarray(cv2.cvtColor(img_pil_or_cv2, cv2.COLOR_BGR2RGB))
    else:
        img = img_pil_or_cv2.convert('RGB')

    # 1. Prioritas tertinggi: MTCNN alignment
    if USE_MTCNN_ALIGN and mtcnn is not None:
        try:
            img_aligned = mtcnn(img)
            if img_aligned is not None:
                img_tensor = img_aligned.unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    emb = model(img_tensor).cpu().numpy()[0]
                return emb
        except:
            pass

    # 2. Fallback: Crop manual dengan Haar + augmentasi berat
    img_cv2 = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    if USE_HAAR_CROP:
        cropped = crop_face_haar(img_cv2)
        if cropped is not None:
            img_cv2 = cropped

    img_pil = Image.fromarray(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))
    aug_img = aug_heavy(image=np.array(img_pil))['image']
    tensor = transform_to_tensor(image=aug_img)['image'].unsqueeze(0).to(DEVICE)
    tensor = tensor.float() / 255.0
    tensor = (tensor - 0.5) / 0.5  # Normalisasi seperti FaceNet

    with torch.no_grad():
        emb = model(tensor).cpu().numpy()[0]
    return emb

## 9. Proses Dataset & Buat Database Embedding

In [9]:
print(f"Mulai proses dataset dengan {AUG_PER_IMAGE}x augmentasi per foto...\n")

database = {}      # {nama: [embeddings]}
val_set = []       # (path, true_label)
test_set = []      # (path, true_label)

for person_name in sorted(os.listdir(DATASET_ROOT)):
    person_path = os.path.join(DATASET_ROOT, person_name)
    if not os.path.isdir(person_path):
        continue

    files = [f for f in os.listdir(person_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    random.shuffle(files)
    train_files = files[:min(9, len(files))]
    val_files   = files[min(9, len(files)):min(12, len(files))]
    test_files  = files[min(12, len(files)):min(14, len(files))]

    print(f"{person_name}: {len(train_files)} train → {len(train_files)*AUG_PER_IMAGE} embeddings | {len(val_files)} val | {len(test_files)} test")

    embeddings = []
    for f in train_files:
        path = os.path.join(person_path, f)
        img_cv2 = cv2.imread(path)
        if img_cv2 is None:
            continue

        # 1 embedding original + AUG_PER_IMAGE augmentasi
        try:
            emb_orig = get_embedding(img_cv2)
            embeddings.append(emb_orig)
            for _ in range(AUG_PER_IMAGE):
                emb_aug = get_embedding(img_cv2)
                embeddings.append(emb_aug)
        except Exception as e:
            print(f"Error pada {f}: {e}")

    if len(embeddings) > 0:
        database[person_name] = np.array(embeddings)

    # Validation & Test (tanpa augmentasi)
    for f in val_files:
        val_set.append((os.path.join(person_path, f), person_name))
    for f in test_files:
        test_set.append((os.path.join(person_path, f), person_name))

print(f"\nTotal orang: {len(database)}")
print(f"Total embeddings train: {sum(len(v) for v in database.values())}")

Mulai proses dataset dengan 50x augmentasi per foto...

Aan Kristian Sitinjak: 9 train → 450 embeddings | 3 val | 2 test
Abeloisa Pardosi: 9 train → 450 embeddings | 3 val | 2 test
Adriano Lumbantoruan: 9 train → 450 embeddings | 3 val | 2 test
Agnes Sidabutar: 9 train → 450 embeddings | 3 val | 2 test
Andri Sigiro: 9 train → 450 embeddings | 3 val | 2 test
Angelika Simamora: 9 train → 450 embeddings | 3 val | 2 test
Anno Siregar: 9 train → 450 embeddings | 3 val | 2 test
Antonia Manalu: 9 train → 450 embeddings | 3 val | 2 test
Arnold Daniel Manalu: 9 train → 450 embeddings | 3 val | 2 test
Ayu Sinaga: 9 train → 450 embeddings | 3 val | 2 test
Bowo Manalu: 9 train → 450 embeddings | 3 val | 2 test
Chelsea Sianipar: 9 train → 450 embeddings | 3 val | 2 test
Chenit Sirait: 9 train → 450 embeddings | 3 val | 2 test
Cheryl: 9 train → 450 embeddings | 3 val | 2 test
Daniel Ginting: 9 train → 450 embeddings | 3 val | 2 test
Diva Marbun: 9 train → 450 embeddings | 3 val | 2 test
Estina Panga

## 10. Simpan Database

In [11]:
names = list(database.keys())
all_train_emb = np.vstack([database[n] for n in names])
all_train_labels = np.array([n for n in names for _ in range(len(database[n]))], dtype=object)

np.save("embeddings.npy", all_train_emb)
np.save("labels.npy", all_train_labels)

print(f"Database berhasil disimpan!")
print(f"→ Embeddings shape: {all_train_emb.shape}")
print(f"→ Labels shape: {all_train_labels.shape}")

Database berhasil disimpan!
→ Embeddings shape: (25245, 512)
→ Labels shape: (25245,)


## 11. Tuning Threshold Otomatis
Mencari threshold cosine similarity terbaik berdasarkan validation set

In [12]:
print("Tuning threshold ultra-presisi...")

def accuracy_at_threshold(th):
    correct = 0
    for path, true_name in val_set:
        try:
            emb = get_embedding(cv2.imread(path))
            sims = np.dot(all_train_emb, emb) / (np.linalg.norm(all_train_emb, axis=1) * np.linalg.norm(emb))
            pred_name = all_train_labels[np.argmax(sims)]
            if sims.max() > th and pred_name == true_name:
                correct += 1
        except:
            pass
    return correct / len(val_set) if val_set else 0

thresholds = np.round(np.arange(0.30, 0.80, 0.002), 3)
accs = [accuracy_at_threshold(th) for th in tqdm(thresholds, desc="Tuning")]

best_th = thresholds[np.argmax(accs)]
best_val_acc = max(accs) * 100

print(f"\nThreshold terbaik: {best_th:.3f}")
print(f"Validation Accuracy: {best_val_acc:.2f}%")

Tuning threshold ultra-presisi...


Tuning: 100%|██████████| 250/250 [1:03:48<00:00, 15.32s/it]


Threshold terbaik: 0.300
Validation Accuracy: 98.79%


## 12. Final Testing + Visualisasi Hasil

In [13]:
print(f"\nFINAL TESTING dengan threshold = {best_th:.3f}")

correct = 0
for path, true_name in tqdm(test_set, desc="Testing"):
    img_bgr = cv2.imread(path)
    if img_bgr is None:
        continue

    emb = get_embedding(img_bgr)
    sims = np.dot(all_train_emb, emb) / (np.linalg.norm(all_train_emb, axis=1) * np.linalg.norm(emb))
    pred_name = all_train_labels[np.argmax(sims)]
    score = sims.max()

    status = "BENAR" if (score > best_th and pred_name == true_name) else "SALAH"
    if status == "BENAR":
        correct += 1

    color = (0, 255, 0) if status == "BENAR" else (0, 0, 255)
    cv2.putText(img_bgr, f"{pred_name}", (10, 50), cv2.FONT_HERSHEY_DUPLEX, 1.8, color, 3)
    cv2.putText(img_bgr, f"True: {true_name}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 2)
    cv2.putText(img_bgr, f"{status} {score:.3f}", (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 1.4, color, 3)

    filename = f"{status}_{os.path.basename(path)}"
    cv2.imwrite(os.path.join(RESULT_FOLDER, filename), img_bgr)

test_acc = correct / len(test_set) * 100
print(f"\nAKURASI TEST FINAL: {correct}/{len(test_set)} = {test_acc:.2f}%")


FINAL TESTING dengan threshold = 0.300


Testing: 100%|██████████| 106/106 [00:10<00:00, 10.26it/s]


AKURASI TEST FINAL: 101/106 = 95.28%


## 13. Simpan Laporan Lengkap

In [14]:
with open(os.path.join(RESULT_FOLDER, "LAPORAN_MAX_ACC.txt"), "w", encoding="utf-8") as f:
    f.write(f"AUG_PER_IMAGE          : {AUG_PER_IMAGE}\n")
    f.write(f"Total orang            : {len(database)}\n")
    f.write(f"Total embeddings       : {all_train_emb.shape}\n")
    f.write(f"Validation Accuracy    : {best_val_acc:.2f}%\n")
    f.write(f"Test Accuracy          : {test_acc:.2f}%\n")
    f.write(f"Threshold optimal      : {best_th:.3f}\n")
    f.write(f"Use MTCNN Alignment    : {USE_MTCNN_ALIGN}\n")
    f.write(f"Dataset path           : {DATASET_ROOT}\n")

print("SELESAI 1000%! yeyyy")
print("\nFile yang dihasilkan:")
print(" • embeddings.npy")
print(" • labels.npy")
print(f" • Folder: {RESULT_FOLDER}/ (gambar hasil + laporan)")

SELESAI 1000%! yeyyy

File yang dihasilkan:
 • embeddings.npy
 • labels.npy
 • Folder: hasil_validasi/ (gambar hasil + laporan)
